In [1]:
import numpy as np
import pandas as pd 
from Utils.ProjectPathsSetup import ProjectPathsSetup
from Utils.Concatenator import Cocatenator
from Environment.PathsParameters import *
from Environment.Parameters import *
from Utils.GraphicPlotter import GraphicPlotter
from sklearn.model_selection import train_test_split
ProjectPathsSetup().add_project_paths('./')

In [2]:

fake_news_df = pd.read_csv(TRUE_NEWS_DATASET)
fake_news_df['label'] = 1
true_news_df = pd.read_csv(FAKE_NEWS_DATASET)
true_news_df['label'] = 0
news_df  = Cocatenator().concatenate(fake_news_df, true_news_df)

In [3]:
#load preprocessed data

#embedding doc
news_embedding_doc = np.loadtxt(PREPROCESSED_DATA_PATH +'embedding_doc_word_news_arrays.csv', delimiter= ',')


#word2vec
news_w2v_encoded = np.loadtxt(PREPROCESSED_DATA_PATH +'w2v_word_news_arrays.csv', delimiter= ',')


#load labels
news_class = np.loadtxt(PREPROCESSED_DATA_PATH + 'labels.csv', delimiter=',')


In [4]:


news_class = news_df['label']

# split train/test text and title embedded doc
X_emb_doc_train, X_emb_doc_test, y_emb_doc_train , y_emb_doc_test = train_test_split(news_embedding_doc, news_class, train_size=0.8, random_state=SEED)


#split train/test text and title  word vectors

X_w2v_train, X_w2v_test, y_w2v_train , y_w2v_test = train_test_split(news_w2v_encoded, news_class, train_size=0.8, random_state=SEED)

In [5]:
from gensim.models import KeyedVectors
from Utils.JSONHandler import JSONHandler

#load Preprocessing Parms
jsonHandler = JSONHandler()
vocab_params = jsonHandler.read_json(PREPROCESSED_DATA_PARAMS_PATH + 'vocab_params.json' )
word_idxs = jsonHandler.read_json(PREPROCESSED_DATA_PARAMS_PATH + 'word_indexes.json' )['word_index_news']
vocab_size = len(word_idxs) + 1

#load google pretrained model 
google_news_word2vec = KeyedVectors.load_word2vec_format(
            GOOGLE_PRETRAINED_MODEL_PATH, 
            binary=True)

gooogle_w2v_emb_mean = google_news_word2vec.vectors.mean()
gooogle_w2v_emb_std = google_news_word2vec.vectors.std()

In [6]:
vocab_size

115317

In [8]:
from DLArchitectures.TCNModel import TCNModel
from DLArchitectures.CNN1D  import CNN1D
from DLArchitectures.EnsembleCNNBiGRU import EnsembleCNNBiGRU
from Environment.Parameters import *

#instantiate and configure TCN
emb_doc_tcn = TCNModel(input_dim= vocab_size)
emb_doc_tcn.build_pretrained_embedding_matrix(google_news_word2vec, word_idxs, gooogle_w2v_emb_mean, gooogle_w2v_emb_std)

#instantiate and configure CNN1D
emb_doc_cnn1d = CNN1D(input_dim= vocab_size, max_length= NEWS_VEC_DIM)
emb_doc_cnn1d.build_pretrained_embedding_matrix(google_news_word2vec, word_idxs, gooogle_w2v_emb_mean, gooogle_w2v_emb_std)

#instantiate and configure EnsembleCNNBiGRU
embdoc_ensemblecnn = EnsembleCNNBiGRU(input_dim= vocab_size, max_length=NEWS_VEC_DIM)
embdoc_ensemblecnn.build_pretrained_embedding_matrix(google_news_word2vec, word_idxs, gooogle_w2v_emb_mean, gooogle_w2v_emb_std)


#get models
emb_doc_tcn_model = emb_doc_tcn.get_model()
emb_doc_cnn1d_model = emb_doc_cnn1d.get_model()
embdoc_ensemblecnn = embdoc_ensemblecnn.get_model()

(115317, 300)


In [16]:
gooogle_w2v_emb_std

0.13315111

In [ ]:
import matplotlib.pyplot as plt
fig = plt.plot([i for i in range(0,30)], [i for i in range(0,30)])
fig.

In [11]:
emb_doc_tcn_model.fit(X_emb_doc_train ,y= y_emb_doc_train, validation_split=0.2, epochs=40, batch_size=32, verbose=1)

Epoch 1/40
 43/898 [>.............................] - ETA: 1:09 - loss: 0.6924 - accuracy: 0.5283

KeyboardInterrupt: 